### New notes taken from latest [tutorial](https://bitbucket.org/hrojas/dc.js-in-ipython-notebooks) on how ipython notebook interacts with dc.js

##Learning Notes

### 1. Use `<iframe>` to host HTML - help resolve the duplicated tag-id issue
### 2. The hosted page is an integrated html, including `<head>` and `<body>`
### 3. Use jinja2 to interplot the strings


The example below uses dc.js for iris data plotting

In [16]:
import pandas as pd
import jinja2 
from IPython.display import display, Javascript, HTML

In [23]:
df = pd.read_csv("data/iris.csv")
df.head(n = 3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower_type
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [19]:
def iframe(stuff):
    return HTML("""<iframe srcdoc='{srcdoc}' style='width:100%; height: 300px; border: none'></iframe>"""
                .format(srcdoc = stuff.replace('"', '&quot;')))

In [24]:
head = HTML("""
<head>
    <link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/dc/2.0.0-alpha.2/dc.css"/> 
    <script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min.js"></script> 
    <script src="https://cdnjs.cloudflare.com/ajax/libs/crossfilter/1.3.9/crossfilter.min.js"></script> 
    <script src="https://cdnjs.cloudflare.com/ajax/libs/dc/1.7.0/dc.min.js"></script> 
</head>
""")

body = HTML("""
<!-- A div anchor that can be identified by id -->
<div id="rowchart" class="dc-chart">
    <!-- Title or anything you want to add above the chart -->
    <strong>Row Chart</strong>
    <!--
        This will create a reset button when ever the user selects a filter.
        Clicking on the reset button will remove all filters.
     -->
    <a class="reset" href="javascript:void(0);" onclick="javascript:RowChart.filterAll();dc.redrawAll();" style="display: none;">reset</a>
    <div class="clearfix"></div>
    <!--
        dc.js will also automatically inject applied current filter value into
        any html element with css class set to "filter"
    -->
    <span class="reset" style="display: none;">Current filter: <span class="filter"></span></span>
    <br>
    </br>
</div>
""")

dc_template = jinja2.Template("""   

// Create Global Variables
var RowChart = dc.rowChart("#rowchart");

// Load data
var dataset = {{ data }};

// Call function
Graph(dataset);


// Create function
function Graph(data) {

   
    // Feed it through crossfilter  
    var ndx = crossfilter(data);
    
    // for testing
    //console.log(data);

    //define a dimension
    //Here we will group by state
    var dim = ndx.dimension(function(d) {return d["flower_type"]});

    //Here we group by state and sum on column population
    var g = dim.group().reduceSum(function(d){return d["sepal width (cm)"];});
    
    //Lets create a row chart
    RowChart.dimension(dim)
            .group(g)
            .width(500);      

    dc.renderAll(); // render all charts on the page
    
}; // end graph function    
    
"""
)

dc_plot = Javascript(dc_template.render(data = df.to_json(orient="records")))

In [25]:
iframe(head.data + body.data + "<script>" + dc_plot.data + "</script>")